## Trabajo práctico 4 - Balance

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

1. Tomar una foto con sus celulares tapando bien el sensor para que no entre nada de luz (se debe obtener una imagen negra, por supuesto....pero no todos los píxeles estarán en cero ya que hay ruido!). Se pide hacer un análisis estadístico del ruido indicando media, desvío y comentando las posibles fuentes del mismo.

2. Tomar una foto también con el celular del patrón de barras generado según el video y obtener el MTF.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline
img_fnames = glob('./fotos/IMG_*')
print(img_fnames) #Agarramos todas las imagenes en el folder fotos

## 1) Análisis estadístico del ruido

Para este punto sacamos 10 fotos con cada celular para poder comparar los sensores. Las fotos fueron tomadas con el sensor de la camara completamente tapado. A continuación se detallan los datos de los celulares

**Xiaomi Redmi Note 8**
* Ancho: 4000 pixeles
* Alto: 3000 pixeles
* Tiempo de exposición: 1/14 seg
* Valor de apertura: 1.67 EV (f/1.8)
* Tasa de velocidad ISO: 9510
* Modo de medida: Promedio ponderado en el centro
* Longitud focal: 4.7 mm


**iPhone 7**
* Ancho: 4032 pixeles
* Alto: 3024 pixeles
* Tiempo de exposición: 1/4 seg
* Valor de apertura: 1.70 EV (f/1.8)
* Tasa de velocidad ISO: 1600
* Modo de medida: Patron
* Longitud focal: 4.0 mm

In [ ]:
imgsXiaomi = []
imgsIphone = []
fname = img_fnames[0] #Para que es esto?
for name in img_fnames:
    img = cv2.imread(name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgsXiaomi.append(img) if 'BURST' in name else imgsIphone.append(img)

print(f"Minima intensidad Xiaomi: {np.min([imgsXiaomi])}")
print(f"Maxima intensidad Xiaomi: {np.max([imgsXiaomi])}\n")

print(f"Minima intensidad Iphone: {np.min([imgsIphone])}")
print(f"Maxima intensidad Iphone: {np.max([imgsIphone])}")

Como se puede ver, en ambos casos la intensidad mínima es cero, pero no sucede lo mismo con la intensidad máxima, lo esperado sería que también sea cero ya que el sensor se encuentra tapado.

In [ ]:
def statistics(imgStack):
    return  np.mean(imgStack, axis=0),  np.std(imgStack, axis=0)

In [ ]:
imgs_np = np.stack(imgsXiaomi)

a = np.mean(imgs_np, axis=0)
b = np.std(imgs_np, axis=0)
#print(f"shape de las imagenes{imgs_np.shape}\nMedia de las imagenes: {a.shape}\nDesvio estandar: {b}")

In [ ]:
imgs_np = np.stack(imgsXiaomi)
stackXiaomi = np.stack(imgsXiaomi)
stackIphone = np.stack(imgsIphone)

# Media y desvio Iphone
img_mediaIphone, img_stdIphone = statistics(stackIphone)

# Media y desvio Xiaomi
img_mediaXiaomi, img_stdXiaomi = statistics(stackXiaomi)

In [ ]:
def dibujar_contorno(mat, title = ''):
    fig = plt.figure()
    X, Y = np.meshgrid(range(len(mat[0])), range(len(mat)))
    Z = mat

    # decimación para no matar la compu calculando contornos!
    dec = 16

    fig = plt.figure(figsize=(16,12))
    cp = plt.contourf(X[::dec], Y[::dec], Z[::dec])
    fig.colorbar(cp)
    plt.title(title, fontsize=18, fontweight='bold')
    plt.show();
    return cp

A continuación vamos a realizar los gráficos de la media y el desvío en los distintos canales para ambos celulares utilizando la función brindada por la cátedra.

In [ ]:
CHANNELS = ['Rojo', 'Verde', 'Azul']
def plotMeanAndStdChannels(meanImgs, stdImgs):
    for chNum, ch in enumerate(CHANNELS):
        dibujar_contorno(meanImgs[:,:,chNum], 'Media ' + ch)
        dibujar_contorno(stdImgs[:,:,chNum], 'Std ' + ch)

### iPhone

In [ ]:
plotMeanAndStdChannels(img_mediaIphone, img_stdIphone)

### Xiaomi

In [ ]:
plotMeanAndStdChannels(img_mediaXiaomi, img_stdXiaomi)

## Estadísticas de ruido: relación entre media y desvío

Para analizar esta relación vamos a realizar histogramas en 2 dimensiones. Como venimos haciendo hasta el momento, lo realizaremos tanto para el iPhone como para el Xiaomi.

In [ ]:
def plotHist2d(meanImgs, stdImgs):
    dec = 100
    for chNum, ch in enumerate(CHANNELS):
        channelStd = np.ravel(stdImgs[:,:,chNum])
        channelMean = np.ravel(meanImgs[:,:,chNum])
        
        #Realizamos el plot del histograma 2d
        plt.figure(figsize=(10,10))
        plt.title(ch, fontweight='bold', fontsize=18)
        plt.xlabel('Media', fontweight='bold', fontsize=16)
        plt.ylabel('Desvio', fontweight='bold', fontsize=16)
        cb = plt.hist2d(channelMean[::dec], channelStd[::dec], bins=100)

### iPhone

In [ ]:
plotHist2d(img_mediaIphone, img_stdIphone)

### Xiaomi

In [ ]:
plotHist2d(img_mediaXiaomi, img_stdXiaomi)

## Histogramas

In [ ]:
def plotHist(stackImgs, title = ''):
    dec = 100
    i_max = 50
    colors = ['red', 'green', 'blue']
    plt.figure(figsize=(10,10))
    plt.title('Histograma ' + title, fontsize=18, fontweight='bold')
    plt.grid()
    for chNum, ch in enumerate(CHANNELS):
        allChannel = np.ravel(imgs_np[:,:,:,chNum])
        _ = plt.hist(allChannel[::dec], bins=range(i_max), color=colors[chNum],histtype='step', linewidth=2.0)


    _ = plt.hist(np.ravel(stackImgs)[::dec], bins=range(i_max), color='black', histtype='step', linewidth=2.0)
    plt.legend(['Rojo', 'Verde', 'Azul', 'Total'])

In [ ]:
plotHist(stackIphone, 'iPhone')

In [ ]:
plotHist(stackXiaomi, 'Xiaomi')

## Conclusiones 1

## 2) MTF

In [ ]:
#Parte mti, f por vos licha

## Conclusiones 2